In [1]:
import time
import pandas as pd # library for data analsysis
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

import json # library to handle JSON files
import requests # library to handle requests
from pandas.io.json import json_normalize # tranform JSON file into a pandas dataframe

from geopy.geocoders import Nominatim # convert an address into latitude and longitude values
!conda install -c conda-forge folium=0.5.0 --yes # uncomment this line if you haven't completed the Foursquare API lab
import folium # map rendering library

print('Got Libraries')

Solving environment: done

# All requested packages already installed.

Got Libraries


In [2]:
address = 'Adyar, Chennai'
geolocator = Nominatim(user_agent="smy-application")
location = geolocator.geocode(address)
latitude_x = location.latitude
longitude_y = location.longitude
print('The Geograpical Co-ordinate of Adyar, Chennai are {}, {}.'.format(latitude_x, longitude_y))

The Geograpical Co-ordinate of Adyar, Chennai are 13.00645, 80.2577791.


In [3]:
neighborhood_latitude=13.00645
neighborhood_longitude=80.2577791

In [4]:
LIMIT = 100 # limit of number of venues returned by Foursquare API
radius = 500
CLIENT_ID='ZWLCLXKRKKBH1AXN2EMN0TLRRVCPDINO1CYW4AVA5VUFANK3'
CLIENT_SECRET='2OKBDY3EH23YLKNIS1UEDB4BHBGBLB2SP4EPOABWQ0LCJF5X'
VERSION='20180605' # Foursquare API version

url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
    CLIENT_ID, 
    CLIENT_SECRET, 
    VERSION, 
    neighborhood_latitude, 
    neighborhood_longitude, 
    radius, 
    LIMIT)
url # display URL

'https://api.foursquare.com/v2/venues/explore?&client_id=ZWLCLXKRKKBH1AXN2EMN0TLRRVCPDINO1CYW4AVA5VUFANK3&client_secret=2OKBDY3EH23YLKNIS1UEDB4BHBGBLB2SP4EPOABWQ0LCJF5X&v=20180605&ll=13.00645,80.2577791&radius=500&limit=100'

In [5]:
results = requests.get(url).json()
results

{'meta': {'code': 200, 'requestId': '60360c8ab9a1cf0a87c1fd4f'},
 'response': {'suggestedFilters': {'header': 'Tap to show:',
   'filters': [{'name': 'Open now', 'key': 'openNow'}]},
  'headerLocation': 'Chennai',
  'headerFullLocation': 'Chennai',
  'headerLocationGranularity': 'city',
  'totalResults': 31,
  'suggestedBounds': {'ne': {'lat': 13.010950004500003,
    'lng': 80.26238896991072},
   'sw': {'lat': 13.001949995499995, 'lng': 80.25316923008927}},
  'groups': [{'type': 'Recommended Places',
    'name': 'recommended',
    'items': [{'reasons': {'count': 0,
       'items': [{'summary': 'This spot is popular',
         'type': 'general',
         'reasonName': 'globalInteractionReason'}]},
      'venue': {'id': '55b9abc8498ecf5798ac0d58',
       'name': 'Bombay Brassiere',
       'location': {'address': 'Adyar',
        'lat': 13.006961450666138,
        'lng': 80.25641939452001,
        'labeledLatLngs': [{'label': 'display',
          'lat': 13.006961450666138,
          'lng'

In [9]:
def get_category_type(row):
    try:
        categories_list = row['categories']
        
    except:
        categories_list = row['venue.categories']
        
    if len(categories_list) == 0:
        return None
    else:
        return categories_list[0]['name']

In [43]:
venues = results['response']['groups'][0]['items']
    
nearby_venues = json_normalize(venues) # flatten JSON

# filter columns
filtered_columns = ['venue.name', 'venue.categories', 'venue.location.lat', 'venue.location.lng']
nearby_venues =nearby_venues.loc[:, filtered_columns]

# filter the category for each row
nearby_venues['venue.categories'] = nearby_venues.apply(get_category_type, axis=1)
nearby_venues.loc[:,'venue.categories'] = 'Vegetarian'
# clean columns
nearby_venues.columns = [col.split(".")[-1] for col in nearby_venues.columns]

nearby_venues.head(10)

/opt/conda/envs/Python-3.7-main/lib/python3.7/site-packages/ipykernel/__main__.py:3: FutureWarning: pandas.io.json.json_normalize is deprecated, use pandas.json_normalize instead
  app.launch_new_instance()


,name,categories,lat,lng
0,Bombay Brassiere,Vegetarian,13.006961,80.256419
1,Cool Biz,Vegetarian,13.006632,80.257275
2,Adyar Ananda Bhavan,Vegetarian,13.005824,80.257368
3,Gangotree,Vegetarian,13.007435,80.258361
4,McDonald's,Vegetarian,13.006632,80.254255
5,Prems Graama Bhojanam,Vegetarian,13.006345,80.253995
6,M'bessy,Vegetarian,13.007361,80.259560
7,Odyssey Book Shop,Vegetarian,13.006682,80.253469
8,Hotel Coronet,Vegetarian,13.006548,80.257307
9,Aavin Milk Booth,Vegetarian,13.005150,80.259490


In [45]:
latitude = neighborhood_latitude 
longitude = neighborhood_longitude
map_ch = folium.Map(location=[latitude, longitude], zoom_start=18)

# add markers to map
for lat, lng, label in zip(nearby_venues['lat'], nearby_venues['lng'], nearby_venues['name']):
    label = folium.Popup(label, parse_html=True)
    folium.RegularPolygonMarker(
        [lat, lng],
        number_of_sides=6,
        radius=12,
        popup=label,
        color='green',
        fill_color='#0f0f0f',
        fill_opacity=0.7,
    ).add_to(map_ch)  
    
map_ch

# <SPAN STYLE="text-decoration:underline">  <span style="color:green">  3.0	Methodology section </span> 
    
    This section represents the main component of the report where the data is gathered, 
    prepared for analysis. The tools described are used here and the Notebook cells indicates the execution of steps.

# <SPAN STYLE="text-decoration:underline">  <span style="color:green">  4.0	Results section </span> 
    Finally we filtered the restaurants having only Vegeterian food those are near by Adyar, Chennai.  

# <SPAN STYLE="text-decoration:underline">  <span style="color:green">  5.0 Discussion section </span> 
    
    Capstone project presented me a great opportunity to practice and apply the Data Science tools and methodologies learned

# <SPAN STYLE="text-decoration:underline">  <span style="color:green">  6.0 Conclusion section </span> 
    
    The mapping with Folium is a very powerful technique to consolidate information and make the analysis 
    and decision thoroughly and with confidence. I would recommend for use in similar situations.
 